## 📀 Локальное подключение Google Диска

In [1]:
!pip install tensorflow-determinism

  Created wheel for tensorflow-determinism: filename=tensorflow_determinism-0.3.0-cp37-none-any.whl size=9158 sha256=67d2c8cc4493350a1e6db1e0e27704ef610beaf3e225f20c7210cb810b77b8ae
  Stored in directory: /root/.cache/pip/wheels/66/c3/18/13959a90d3e0d10182a99866d6ff4d0119e9daed6ce014b54c
Successfully built tensorflow-determinism


In [2]:
import warnings
warnings.filterwarnings("ignore")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import os

# Для воспроизводимости вычислений
random_seed = 123
os.environ['TF_CUDNN_DETERMINISTIC'] = 'true'
os.environ['TF_CUDNN_USE_AUTOTUNE'] = 'false'
os.environ['HOROVOD_FUSION_THRESHOLD'] = '0'
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['PYTHONHASHSEED']=str(random_seed)

import math
import random
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import tensorflow as tf
print("Tensorflow version = {}".format(tf.__version__)) # текущая версия tf
import tensorflow_hub as hub
from tensorflow.keras import Model, optimizers, metrics
from tensorflow.keras.layers import Layer, Flatten, Dense, BatchNormalization, Dropout
from sklearn.metrics import roc_auc_score, average_precision_score

# Sets the default float type
tf.keras.backend.set_floatx('float64')

# Sets random seed
random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
tf.compat.v1.set_random_seed(random_seed)

Tensorflow version = 2.4.1


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls

drive  sample_data


## 🔮 Информация о GPU

In [5]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed Mar 24 17:24:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 📁 Читаем датасет

In [6]:
df = pd.read_csv("drive/My Drive/lenta-ru-news.csv")

# df = pd.read_csv("/content/drive/MyDrive/Datasets/data_from_Sait/lenta-ru-news.csv")

In [7]:
df.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


## 🎨 Выбираем темы

In [8]:
topics_list = df['topic'].unique() # получаем список тем
topics_lengths = [len(df['text'][(df['topic']==t)].reset_index(drop=True)) \
                  for t in topics_list]

print("🎨 Тема - 🎰 Количество документов в теме:\n")
for t, len_t in zip(topics_list, topics_lengths):
  print('{} - {}'.format(t, len_t))

🎨 Тема - 🎰 Количество документов в теме:

Библиотека - 65
Россия - 160445
Мир - 136621
Экономика - 79528
Интернет и СМИ - 44663
Спорт - 64413
Культура - 53797
Из жизни - 27605
Силовые структуры - 19596
Наука и техника - 53136
Бывший СССР - 53402
nan - 0
Дом - 21734
Сочи - 1
ЧМ-2014 - 2
Путешествия - 6408
Ценности - 7766
Легпром - 114
Бизнес - 7399
МедНовости - 1
Оружие - 3
69-я параллель - 1268
Культпросвет  - 340
Крым - 666


Выберем темы, которые наиболее отличаются между собой и при этом содержат достаточное количество документов:


1.   Россия
2.   Экономика
3.   Спорт
4.   Культура
5.   Наука и техника
6.   Путешествия
7.   Силовые структуры

Составим словарь из этих тем. В каждую тему возьмем 6400 документов, так как в теме *Путешествия* всего 6408 документов.




In [9]:
list_selecting_topics = [
                         ('Культура', 'Книги'),
                         ('Наука и техника', 'Оружие'),
                         ('Культура', 'Театр'),
                         ('Силовые структуры', 'Полиция и спецслужбы'),
                         ('Наука и техника', 'Гаджеты'),
                         ('Спорт', 'Бокс и ММА'),
                         ('Наука и техника', 'Космос'),
                         ('Наука и техника', 'Наука'),
                         ('Мир', 'Политика'),
                         ('Экономика', 'Госэкономика')
]

topics_dict = {}
for t in list_selecting_topics:
  topic, tag = t[0], t[1]
  tmp_df = df[df['topic']==topic]
  tmp_df = tmp_df[tmp_df['tags']==tag]
  topics_dict[topic + '-' + tag] = list(tmp_df['text'].reset_index(drop=True)[:1000])

In [10]:
normal_topics_list = [
                      'Культура-Театр',
                      'Культура-Театр', 
                      'Экономика-Госэкономика', 
                      'Наука и техника-Гаджеты', 
                      'Наука и техника-Космос', 
                      'Наука и техника-Гаджеты',
                      'Экономика-Госэкономика',
                      'Спорт-Бокс и ММА',
                      'Спорт-Бокс и ММА',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Наука и техника-Космос',
                      'Культура-Театр',
                      'Культура-Книги',
                      'Наука и техника-Оружие',
                      'Наука и техника-Оружие',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Наука и техника-Оружие',
                      'Культура-Театр',
                      'Мир-Политика',
                      'Наука и техника-Гаджеты',
                      'Культура-Книги',
                      'Наука и техника-Оружие',
                      'Наука и техника-Гаджеты',
                      'Экономика-Госэкономика',
                      'Наука и техника-Космос',
                      'Спорт-Бокс и ММА',
                      'Экономика-Госэкономика',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Культура-Театр',
                      'Спорт-Бокс и ММА',
                      'Мир-Политика',
                      'Наука и техника-Оружие',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Экономика-Госэкономика',
                      'Культура-Книги',
                      'Культура-Книги',
                      'Культура-Театр',
                      'Мир-Политика',
                      'Спорт-Бокс и ММА',
                      'Культура-Театр',
                      'Мир-Политика',
                      'Наука и техника-Гаджеты',
                      'Наука и техника-Космос',
                      'Экономика-Госэкономика',
                      'Наука и техника-Оружие',
                      'Спорт-Бокс и ММА',
                      'Спорт-Бокс и ММА',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Наука и техника-Космос',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Культура-Театр',
                      'Спорт-Бокс и ММА',
                      'Спорт-Бокс и ММА',
                      'Наука и техника-Гаджеты',
                      'Культура-Книги',
                      'Мир-Политика',
]

In [11]:
anomal_topics_list = [
                      'Экономика-Госэкономика',
                      'Наука и техника-Гаджеты',
                      'Культура-Театр',
                      'Культура-Театр',
                      'Культура-Театр',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Спорт-Бокс и ММА',
                      'Наука и техника-Гаджеты',
                      'Экономика-Госэкономика',
                      'Наука и техника-Гаджеты',
                      'Культура-Книги',
                      'Культура-Книги',
                      'Наука и техника-Оружие',
                      'Культура-Книги',
                      'Экономика-Госэкономика',
                      'Наука и техника-Наука',
                      'Наука и техника-Гаджеты',
                      'Наука и техника-Оружие',
                      'Наука и техника-Гаджеты',
                      'Мир-Политика',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Культура-Театр',
                      'Наука и техника-Космос',
                      'Наука и техника-Гаджеты',
                      'Мир-Политика',
                      'Культура-Книги',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Экономика-Госэкономика',
                      'Наука и техника-Космос',
                      'Наука и техника-Наука',
                      'Культура-Театр',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Культура-Театр',
                      'Наука и техника-Космос',
                      'Наука и техника-Космос',
                      'Наука и техника-Наука',
                      'Наука и техника-Наука',
                      'Наука и техника-Космос',
                      'Наука и техника-Космос',
                      'Спорт-Бокс и ММА',
                      'Наука и техника-Наука',
                      'Наука и техника-Оружие',
                      'Спорт-Бокс и ММА',
                      'Наука и техника-Наука',
                      'Спорт-Бокс и ММА',
                      'Наука и техника-Оружие',
                      'Культура-Театр',
                      'Спорт-Бокс и ММА',
                      'Наука и техника-Гаджеты',
                      'Культура-Книги',
                      'Мир-Политика',
                      'Мир-Политика',
                      'Силовые структуры-Полиция и спецслужбы',
                      'Спорт-Бокс и ММА',
                      'Наука и техника-Гаджеты',
                      'Наука и техника-Оружие',
]

## 🇷🇺 Universal Sentence Encoder (rus)

In [12]:
!pip install tensorflow_text
import tensorflow_text

     |████████████████████████████████| 3.4MB 4.3MB/s 


In [13]:
hub_layer_rus = hub.KerasLayer(
    'https://tfhub.dev/google/universal-sentence-encoder-multilingual/3',
    input_shape=[], 
    dtype=tf.string,
    trainable=True)

## 📐 Функция Cosine similarity

In [14]:
def cosine_similarity(x_predict, x):
    if type(x_predict) is np.ndarray:
        flat_output = x_predict
        flat_input = x_predict
        # flat_output = np.reshape(x_predict, (np.shape(x)[0], -1))
        # flat_input = np.reshape(x_predict, (np.shape(x)[0], -1))
        sum = np.sum(flat_output * flat_input, -1)
        norm1 = np.linalg.norm(flat_output, axis=-1) + 0.000001
        norm2 = np.linalg.norm(flat_input, axis=-1) + 0.000001 
        return -(sum / norm1 / norm2)
    else:
        # ДЛЯ НЕ ПОЛНОСВЯЗНЫХ СЛОЕВ НУЖЕН ДРУГОЙ shape
        flat_output = x_predict
        flat_input = x_predict
        # flat_output = tf.reshape(tensor=x_predict, shape=[x.shape.as_list()[0], -1])
        # flat_input = tf.reshape(tensor=x_predict, shape=[x.shape.as_list()[0], -1])
        sum = tf.math.reduce_sum(tf.math.multiply(flat_output, flat_input), axis=-1)
        norm1 = tf.norm(flat_output, axis=-1) + 0.000001
        norm2 = tf.norm(flat_input, axis=-1) + 0.000001
        return -(tf.math.divide(tf.math.divide(sum, norm1), norm2))

## 🔫 RSRAE model

In [15]:
class RSR(Layer):
    """
    Robust Subspace Recovery (RSR) layer.
    Робастный слой, восстанавливающий подпространство. Задача данного слоя - отобразить
    закодированные энкодером данные в подпростраство так, чтобы после их обратного
    отображения декодером дивергенция между экземпляром исходных данных и его образом,
    полученным от автоэнкодера была незначительной для нормального экземпляра и была
    большой для аномального экземпляра. 

    # Example
    ```
        z_rsr, A = RSR(intrinsic_size=10)(z)
    ```
    # Arguments
        intrinsic_size: размерность z_rsr.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)` after encoding.
    # Output shape
        2D tensor with shape: `(n_samples, intrinsic_size)`.
    """

    def __init__(self, intrinsic_size: int, name="RSR_layer", **kwargs):
        super(RSR, self).__init__(name=name, **kwargs)
        # Если присваивать экземпляр слоя, как атрибут другого слоя, то хорошей
        # практикой делать создавать такие подслои в __init__ (поскольку подслои обычно
        # имеют метод build, они будут собраны, когда будет собран внешний слой). 
        self.flatten = Flatten()
        self.intrinsic_size = intrinsic_size
        
    def build(self, input_shape):
        """Определяет веса слоя, а именно задает матрицу A."""
        self.A = self.add_weight(name="A",
                                 shape=[int(input_shape[-1]), self.intrinsic_size],
                                 initializer='random_normal',
                                 trainable=True,)
        
        # self.V = self.add_weight(name="V",
        #                          shape=[int(input_shape[-1]), 1],
        #                          initializer='random_normal',
        #                          trainable=True,)
        
    def call(self, z):
        """
        Логика слоя. Умножение выхода энкодера - вектора z на матрицу A.
        Возвращает отображенный z_rsr и матрицу A, которая потребуется далее.
        """
        z = self.flatten(z)
        # print("z.shpae Before A:", z.shape)
        z_rsr = tf.linalg.matmul(z, self.A)
        # print("z_rsr.shpae After A:", z_rsr.shape) 
        return z_rsr

    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        config.update({'intrinsic_size': self.intrinsic_size})
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class L2Normalization(Layer):
    """Слой для l_2 нормализации, который будет применяться к выходу RSR layer."""
    
    def __init__(self, name="L2Normalization", **kwargs):
        super(L2Normalization, self).__init__(name=name, **kwargs)

    def call(self, z_rsr):
        """
        Выполняет l_2 нормализацию векторов, полученных после применения RSR layer
        вдоль оси, соответсвующей числу признаков. То есть производится нормализация
        каждого экземпляра выборки, в результате которой признаки экземпляров будут
        находиться в отрезке [-1; 1].
        """
        z_tilde = tf.math.l2_normalize(z_rsr, axis=-1)
        return z_tilde

    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class Encoder(Layer):
    """
    Класс для encoder модели RSRAE. Отображает исходные данные input_data в вектор z,
    кодирующий исходные данные.
    """

    def __init__(self,
                 hidden_layer_dimensions,
                 activation,
                 flag_bn=True, 
                 name="Encoder",
                 **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.hidden_layer_dimensions = hidden_layer_dimensions
        self.activation = activation
        self.flag_bn = flag_bn
        self.dense0 = Dense(hidden_layer_dimensions[0], activation=activation,
                            name='encoder_0')
        self.dense1 = Dense(hidden_layer_dimensions[1], activation=activation,
                            name='encoder_1')
        self.dense2 = Dense(hidden_layer_dimensions[2], activation=activation,
                            name='encoder_2')
        if flag_bn:
            self.batch_normalization0 = BatchNormalization(name="encoder_bn_layer_0")
            self.batch_normalization1 = BatchNormalization(name="encoder_bn_layer_1")
            self.batch_normalization2 = BatchNormalization(name="encoder_bn_layer_2")

    def call(self, inputs):
        """Отображние исходных данных x -> в закодированный вектор z."""
        x = inputs
        x = self.dense0(x)
        if self.flag_bn:
            x = self.batch_normalization0(x)
        x = self.dense1(x)
        if self.flag_bn:
            x = self.batch_normalization1(x)
        x = Dropout(0.2)((x))
        x = self.dense2(x)
        if self.flag_bn:
            x = self.batch_normalization2(x)
        x = Dropout(0.2)((x))    
        z = x
        return z
    
    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        config.update({'hidden_layer_dimensions': self.hidden_layer_dimensions})
        config.update({'activation': self.activation})
        config.update({'flag_bn': self.flag_bn})
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class Decoder(Layer):
    """
    Класс для decoder модели RSRAE. Отображает вектор z_rsr, полученный в результате
    кодирования исходных данных в вектор z, и последующим отображением вектора z при
    помощи RSR layer (x -> z -> z_rsr), обратно в пространство исходных данных 
    (z_rsr -> x_tilde).
    """

    def __init__(self,
                 inputs_dim,
                 hidden_layer_dimensions,
                 activation,
                 flag_bn=True, 
                 name="Decoder",
                 **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.hidden_layer_dimensions = hidden_layer_dimensions
        self.activation = activation
        self.flag_bn = flag_bn
        self.dense2 = Dense(hidden_layer_dimensions[2], activation=activation,
                            name='decoder_2')
        self.dense1 = Dense(hidden_layer_dimensions[1], activation=activation,
                            name='decoder_1')
        self.dense0 = Dense(hidden_layer_dimensions[0], activation=activation,
                            name='decoder_0')
        self.dense_output = Dense(inputs_dim, activation=activation,
                            name='decoder_output')
        if flag_bn:
            self.batch_normalization2 = BatchNormalization(name="decoder_bn_layer_2")
            self.batch_normalization1 = BatchNormalization(name="decoder_bn_layer_1")
            self.batch_normalization0 = BatchNormalization(name="decoder_bn_layer_0")
    def call(self, inputs):
        """
        Отображние z_rsr -> x_tilde, где x_tilde - вектор, лежащий в пространстве
        исходных даных.
        """
        z_rsr = inputs
        z_rsr = self.dense2(z_rsr)
        if self.flag_bn:
            z_rsr = self.batch_normalization2(z_rsr)
        z_rsr = Dropout(0.3)((z_rsr))
        z_rsr = self.dense1(z_rsr)
        if self.flag_bn:
            z_rsr = self.batch_normalization1(z_rsr)
        z_rsr = Dropout(0.4)((z_rsr))
        z_rsr = self.dense0(z_rsr)
        if self.flag_bn:
            z_rsr = self.batch_normalization0(z_rsr)
        z_rsr = Dropout(0.4)((z_rsr))
        x_tilde = self.dense_output(z_rsr)
        return x_tilde
    
    # Опционально, пользовательский слой может быть сериализован реализацией метода 
    # get_config и метода класса (@classmethod) from_config.
    def get_config(self):
        config = super(Layer, self).get_config()
        config.update({'hidden_layer_dimensions': self.hidden_layer_dimensions})
        config.update({'activation': self.activation})
        config.update({'flag_bn': self.flag_bn})
        return config

    # На самом деле нет необходимости определять `from_config` здесь, поскольку 
    # возвращение `cls(**config)` - поведение по умолчанию.
    @classmethod
    def from_config(cls, config):
        return cls(**config)


class RSRAE(Model):
    """
    Нейросетевая модель-автоэнкодер для обнаружения аномалий с робастным слоем,
    восстанавливающим подпространство (RSR layer между encoder и decoder).
    Комбинируем encoder + RSR layer + decoder в end-to-end модель.
    """

    def __init__(self,
                 inputs_dim, # размерность вектора признаков
                 hidden_layer_dimensions,
                 intrinsic_size, # разерность z_rsr после RSR layer
                 activation,
                 hub_layer,
                 flag_loss,
                 flag_bn=True,
                 flag_normalize=True,
                 learning_rate=1e-3,
                 beta=1,
                 eta=1,
                 t_step=0,
                 ae_loss_norm_type='MSE',
                 rsr_loss_norm_type='MSE',
                 name='RSRAE',
                 **kwargs):
        super(RSRAE, self).__init__(name=name, **kwargs)
        self.inputs_dim = inputs_dim
        self.hidden_layer_dimensions = hidden_layer_dimensions
        self.intrinsic_size = intrinsic_size
        self.activation = activation
        self.flag_bn = flag_bn
        self.flag_normalize = flag_normalize
        self.flag_loss = flag_loss
        self.learning_rate = learning_rate
        self.beta = tf.Variable(beta, dtype=tf.float64, trainable=False)
        self.beta0 = tf.Variable(beta, dtype=tf.float64, trainable=False)
        self.eta = tf.Variable(eta, dtype=tf.float64, trainable=False)
        self.eta0 = tf.Variable(eta, dtype=tf.float64, trainable=False)
        self.t_step = tf.Variable(t_step, dtype=tf.float64, trainable=False)
        self.ae_loss_norm_type = ae_loss_norm_type
        self.rsr_loss_norm_type = rsr_loss_norm_type
        # Для вычисления среднего loss по loss всех батчей в эпохе
        self.loss_tracker = metrics.Mean(name="loss")
        self.auc_tracker = metrics.Mean(name="auc")
        self.ap_tracker = metrics.Mean(name="ap")

        # Создание экземпляров оптимизаторов
        self.optimizer_ae = optimizers.Adam(learning_rate=learning_rate)
        self.optimizer_rsr1 = optimizers.Adam(learning_rate=5 * learning_rate)
        self.optimizer_rsr2 = optimizers.Adam(learning_rate=5 * learning_rate)

        # Слои
        self.emnedding_layer = hub_layer
        self.encoder = Encoder(hidden_layer_dimensions=hidden_layer_dimensions,
                               activation=activation,
                               flag_bn=flag_bn)
        self.rsr = RSR(intrinsic_size=intrinsic_size)
        if flag_normalize:
            self.l2normalization = L2Normalization()
        self.decoder = Decoder(inputs_dim=inputs_dim,
                               hidden_layer_dimensions=hidden_layer_dimensions,
                               activation=activation,
                               flag_bn=flag_bn)
        
    def call(self, inputs):
        e = self.emnedding_layer(inputs)
        e = tf.cast(e, dtype=tf.float64)
        z = self.encoder(e)
        z_rsr = self.rsr(z)
        if self.flag_normalize:
            z_rsr = self.l2normalization(z_rsr)
        x_tilde = self.decoder(z_rsr)
        return e, z, z_rsr, x_tilde

    def ae_loss(self, x, x_tilde):
        """Функция потерь реконструкции автоэнкодера - L_AE."""

        x = tf.reshape(x, (tf.shape(x)[0], -1))
        x_tilde = tf.reshape(x_tilde, (tf.shape(x_tilde)[0], -1))

        # axis=1 для tf.norm => вычисление вдоль оси признаков
        # tf.math.reduce_mean без параметров - mean от элементов матрицы
        if self.ae_loss_norm_type in ['MSE', 'mse', 'Frob', 'F']:
            return tf.math.reduce_mean(tf.math.square(tf.norm(x-x_tilde, 
                                                              ord=2, axis=1)))
        elif self.ae_loss_norm_type in ['L1', 'l1']:
            return tf.math.reduce_mean(tf.norm(x-x_tilde, ord=1, axis=1))
        elif self.ae_loss_norm_type in ['LAD', 'lad', 'L21', 'l21', 'L2', 'l2']:
            return tf.math.reduce_mean(tf.norm(x-x_tilde, ord=2, axis=1))
        else:
            raise Exception("Norm type error!")
    
    def rsr1_loss(self, z, z_rsr, beta, eta):
        """Функция потери для RSR layer - L_RSR1."""
        z_rsr = tf.matmul(z_rsr, tf.transpose(self.rsr.A))
        # z_rsr_new = tf.matmul(z_rsr, self.)

        if self.rsr_loss_norm_type in ['MSE', 'mse', 'Frob', 'F']:
            return tf.math.reduce_mean(tf.math.square(tf.norm(z-z_rsr, ord=2, 
                                                            axis=1)))
        elif self.rsr_loss_norm_type in ['L1', 'l1']:
            return tf.math.reduce_mean(tf.norm(z-z_rsr, ord=1, axis=1))
        elif self.rsr_loss_norm_type in ['LAD', 'lad', 'L21', 'l21', 'L2', 'l2']:
            return tf.math.reduce_mean(tf.norm(z-z_rsr, ord=2, axis=1))
        else:
            raise Exception("Norm type error!")
    
    def rsr2_loss(self):
        """Функция потери для RSR layer - L_RSR2."""
        A = self.rsr.A
        A_T = tf.transpose(A)
        I = tf.eye(self.intrinsic_size, dtype=tf.float64)
        return tf.math.reduce_mean(tf.math.square(tf.linalg.matmul(A_T, A) - I))

    def rsr3_loss(self, z, z_rsr, beta, eta):
        """
        Cтатьи 'Robust principal component analysis by 
        self-organizing rules basedon statistical physics approach', на которую
        ссылается http://files.is.tue.mpg.de/black/papers/delatorreIJCV03.pdf
        """
        z_rsr = tf.matmul(z_rsr, tf.transpose(self.rsr.A))  # AA'z
        e_pca = tf.math.square(tf.norm(z-z_rsr, ord=2, axis=1))
        # self.min_div = tf.reduce_min(e_pca)
        # self.max_div = tf.reduce_max(e_pca)
        # self.mean_div = tf.reduce_mean(e_pca)
        loss = -1 * tf.math.reduce_mean(tf.math.log(1 + tf.math.exp(-beta * e_pca - eta))) / beta
        return loss
        
    def gradients(model, inputs, targets):
        with tf.GradientTape() as tape:
            loss_value = loss_fn(model, inputs, targets)
        return tape.gradient(loss_value, model.trainable_variables)
    
    @tf.function()
    def train_step(self, data):
        """
        Override the method. Будет вызываться при 'model.fit()'.
        Один шаг обучения, на котором вычисляются функции потерь для автоэнкодера и
        RSR layer, и в соотвествии с ними обновляются значения обучаемых переменных - 
        весов нейросети и матрицы A соответсвенно. Будет вызываться от одного батча.
        Заметим, что в этом методе мы используем пользовательские оптимизаторы и функции
        потерь, поэтому перед тренировкой метод compile вызывать не придется.
        """


        x, y = data

        # tf.GradientTape() - записывает операции для автоматического дифференцирования

        # По умолчанию persistent=False и удерживаемые GradientTape, высвобождаются,
        # как только вызывается метод GradientTape.gradient(). Чтобы вычислить несколько
        # градиентов за одно вычисление, требуется задать persistent=true. Это позволяет
        # многократно вызывать метод gradient(), тогда требуется самостоятельно
        # освободить ресурсы с помощью 'del tape'.

        # watch_accessed_variables=True => автоматическое отслеживание всех обучаемых
        # переменные, к которым осуществляется доступ. Так градиенты могут быть
        # запрошены c любого вычисленного результата в tape.
        with tf.GradientTape(persistent=True, watch_accessed_variables=True) as tape:
            # Здесь требуется запустить прямой проход нейросети. Операции применяемые
            # при проходе к входных данным будут записаны на GradientTape. 
            e, z, z_rsr, x_tilde = self.call(x) # прямой проход RSRAE
            z = tf.keras.layers.Flatten()(z) # вроде для текстовых данных необязательно
            # Вычисляем значения функций потерь для этого прохода
            loss_ae = self.ae_loss(e, x_tilde)
            if (self.flag_loss == 0):
                loss_rsr3 = self.rsr1_loss(z, z_rsr, self.beta, self.eta)
            else:
                loss_rsr3 = self.rsr3_loss(z, z_rsr, self.beta, self.eta)
            loss_rsr2 = self.rsr2_loss()
  
        # Метод gradient вычисляет градиенты обучаемых параметров(весов) для минимизации
        # функции потерь, используя операции, записанные в контексте этого tape.
        gradients_ae = tape.gradient(loss_ae, self.trainable_weights)
        gradients_rsr3 = tape.gradient(loss_rsr3, self.rsr.A)
        gradients_rsr2 = tape.gradient(loss_rsr2, self.rsr.A)

        # Обновим значения обучаемых переменных - градиентный шаг чтобы min loss.
        self.optimizer_ae.apply_gradients(grads_and_vars=
                                          zip(gradients_ae, self.trainable_weights))
        self.optimizer_rsr1.apply_gradients(grads_and_vars=
                                            zip([gradients_rsr3], [self.rsr.A]))
        self.optimizer_rsr2.apply_gradients(grads_and_vars=
                                            zip([gradients_rsr2], [self.rsr.A]))
        
        self.loss_tracker.update_state(loss_ae) # обновляем средний loss по батчам

        self.t_step.assign_add(1, use_locking=True)
        self.beta.assign(self.beta0.value() * tf.math.log(self.t_step.value() + 3))
        self.eta.assign(self.eta0.value() * self.t_step.value())

        # Обновляем метрики
        if len(tf.unique(y)[0]) == 2:
            # иначе roc_auc_score бросит ValueError и обучение приостановится
            auc = self.auc_metric(y, cosine_similarity(x_tilde, e))
            self.auc_tracker.update_state(auc)

        ap = self.ap_metric(y, cosine_similarity(x_tilde, e))
        self.ap_tracker.update_state(ap)

        del tape # persistent=True => требуется самостоятельно освободить ресурсы
        return {"loss": self.loss_tracker.result(),
                "auc": self.auc_tracker.result(),
                "ap": self.ap_tracker.result(),
                # "mean_div": self.mean_div,
                # "min_div": self.min_div, 
                # "max_div": self.max_div,
                "beta": self.beta,
                "eta": self.eta,
                "t_step": self.t_step}

    @property
    def metrics(self):
        """
        В пару к train_step. Сбрасывает метрики (`reset_states()`) в начале каждой
        эпохи обучения с помощью 'fit()'. Без этого свойства 'result()' будет 
        возвращать среднее значение с начала обучения.
        """
        return [self.loss_tracker, self.auc_tracker, self.ap_tracker]

    def auc_metric(self, y_true, y_pred):
        return tf.py_function(roc_auc_score, (y_true, y_pred), tf.float64)

    def ap_metric(self, y_true, y_pred):
        return tf.py_function(average_precision_score, (y_true, y_pred), tf.float64)

## 🧫 Эксперименты

In [16]:
# import numpy as np
# from sklearn.utils import shuffle
# import pandas as pd

# t1 = 'Культура-Книги'
# t2 = 'Силовые структуры-Полиция и спецслужбы'


# # Формирование выборок
# normal_data = topics_dict[t1]
# anomal_data = topics_dict[t2][:int(c * len(normal_data))]
# all_data = normal_data + anomal_data
# x = pd.Series(all_data)
# y = np.array([False] * len(normal_data) + [True] * len(anomal_data))
# all_data, x, y = shuffle(all_data, x, y, random_state=123)

# # Изначальный слой USE
# layer = hub_layer_rus
# config = layer.get_config()
# weights = layer.get_weights()
# cloned_layer = type(layer).from_config(config)
# cloned_layer.set_weights(weights)
# hub_layer_now = cloned_layer

# # Тренировка модели (новый лосс)
# model_rsrae = RSRAE(inputs_dim=512,
#                     hidden_layer_dimensions=[512, 1024, 2048],
#                     intrinsic_size=20,
#                     activation='relu',
#                     hub_layer=hub_layer_now,
#                     flag_loss=1, # новый лосс
#                     learning_rate=1e-5,
#                     beta=1.0,
#                     eta=0.0015,
#                     ae_loss_norm_type='MSE',
#                     rsr_loss_norm_type='MSE',)
# model_rsrae.compile(run_eagerly=True)
# model_rsrae.fit(x, y,
#                 batch_size=128,
#                 epochs=17)

# e, _, _, x_predict = model_rsrae.call(x)
# auc1 = roc_auc_score(y, cosine_similarity(x_predict, e))
# print(auc1)

In [17]:
c = 0.1  # отношение количества аномальных экземпляров к нормальным

experimant_cnt = 0
all_experiments = len(normal_topics_list)
auc_list0 = []
auc_list1 = []

# Перебираем пары тем
for t1, t2 in zip(normal_topics_list, anomal_topics_list):
    if t1 == t2:
      continue

    experimant_cnt += 1

    # Формирование выборок
    normal_data = topics_dict[t1]
    anomal_data = topics_dict[t2][:int(c * len(normal_data))]
    all_data = normal_data + anomal_data
    x = pd.Series(all_data)
    y = np.array([False] * len(normal_data) + [True] * len(anomal_data))
    all_data, x, y = shuffle(all_data, x, y, random_state=123)

    # Изначальный слой USE
    layer = hub_layer_rus
    config = layer.get_config()
    weights = layer.get_weights()
    cloned_layer = type(layer).from_config(config)
    cloned_layer.set_weights(weights)
    hub_layer_now = cloned_layer

    # Тренировка модели (новый лосс)
    model_rsrae = RSRAE(inputs_dim=512,
                        hidden_layer_dimensions=[512, 1024, 2048],
                        intrinsic_size=20,
                        activation='relu',
                        hub_layer=hub_layer_now,
                        flag_loss=1, # новый лосс
                        learning_rate=1e-5,
                        beta=1.0,
                        eta=0.0015,
                        ae_loss_norm_type='MSE',
                        rsr_loss_norm_type='MSE',)
    model_rsrae.compile(run_eagerly=True)
    model_rsrae.fit(x, y,
                    batch_size=128,
                    epochs=8,
                    verbose=0)
    
    e, _, _, x_predict = model_rsrae.call(x)
    auc1 = roc_auc_score(y, cosine_similarity(x_predict, e))
    auc_list1.append(auc1)

    # Изначальный слой USE
    layer = hub_layer_rus
    config = layer.get_config()
    weights = layer.get_weights()
    cloned_layer = type(layer).from_config(config)
    cloned_layer.set_weights(weights)
    hub_layer_now = cloned_layer


    # Тренировка модели (старый лосс)
    model_rsrae = RSRAE(inputs_dim=512,
                        hidden_layer_dimensions=[512, 1024, 2048],
                        intrinsic_size=20,
                        activation='relu',
                        hub_layer=hub_layer_now,
                        flag_loss=0, # старый лосс
                        learning_rate=1e-5,
                        beta=1.0,
                        eta=0.0015,
                        ae_loss_norm_type='MSE',
                        rsr_loss_norm_type='MSE',)
    model_rsrae.compile(run_eagerly=True)
    model_rsrae.fit(x, y,
                    batch_size=128,
                    epochs=8,
                    verbose=0)
    
    e, _, _, x_predict = model_rsrae.call(x)
    auc = roc_auc_score(y, cosine_similarity(x_predict, e))
    auc_list0.append(auc)

    print("-" * 50)
    print("Эксперимент №{}/{}  с normal = {}, anomal = {}".format(
        experimant_cnt, all_experiments, t1, t2))
    print("old auc = {}".format(auc))
    print("new auc = {}".format(auc1))
    print("-" * 50)

--------------------------------------------------
Эксперимент №1/56  с normal = Культура-Театр, anomal = Экономика-Госэкономика
old auc = 0.95584
new auc = 0.9573800000000001
--------------------------------------------------


--------------------------------------------------
Эксперимент №2/56  с normal = Культура-Театр, anomal = Наука и техника-Гаджеты
old auc = 0.98457
new auc = 0.99137
--------------------------------------------------


--------------------------------------------------
Эксперимент №3/56  с normal = Экономика-Госэкономика, anomal = Культура-Театр
old auc = 0.84394
new auc = 0.8442799999999999
--------------------------------------------------


--------------------------------------------------
Эксперимент №4/56  с normal = Наука и техника-Гаджеты, anomal = Культура-Театр
old auc = 0.7629
new auc = 0.93286
--------------------------------------------------


--------------------------------------------------
Эксперимент №5/56  с normal = Наука и техника-Космос, anomal = Культура-Театр
old auc = 0.90375
new auc = 0.91325
--------------------------------------------------


...

--------------------------------------------------
Эксперимент №52/56  с normal = Спорт-Бокс и ММА, anomal = Мир-Политика
old auc = 0.96477
new auc = 0.89704
--------------------------------------------------


--------------------------------------------------
Эксперимент №53/56  с normal = Спорт-Бокс и ММА, anomal = Силовые структуры-Полиция и спецслужбы
old auc = 0.9695100000000001
new auc = 0.9912000000000001
--------------------------------------------------


--------------------------------------------------
Эксперимент №54/56  с normal = Наука и техника-Гаджеты, anomal = Спорт-Бокс и ММА
old auc = 0.97184
new auc = 0.9992500000000001
--------------------------------------------------


--------------------------------------------------
Эксперимент №55/56  с normal = Культура-Книги, anomal = Наука и техника-Гаджеты
old auc = 0.9639300000000001
new auc = 0.98932
--------------------------------------------------


--------------------------------------------------
Эксперимент №56/56  с normal = Мир-Политика, anomal = Наука и техника-Оружие
old auc = 0.91659
new auc = 0.88593
--------------------------------------------------


In [51]:
for i in range(len(auc_list0)):
  print(auc_list0[i], auc_list1[i]) 

0.95584 0.9573800000000001
0.98457 0.99137
0.84394 0.8442799999999999
0.7629 0.93286
0.90375 0.91325
0.79228 0.9720300000000002
0.99944 0.99783
0.99908 0.99974
0.98841 0.99119
0.89004 0.83018
0.98871 0.98041
0.90687 0.9786199999999999
0.61542 0.91871
0.91898 0.9654
0.92307 0.9323099999999999
0.9928899999999999 0.9945
0.97338 0.9816199999999999
0.79132 0.83662
0.74452 0.77513
0.98018 0.95355
0.8390599999999999 0.93571
0.9818899999999999 0.96351
0.8693299999999999 0.9557
0.77412 0.85666
0.99177 0.99438
0.98813 0.99631
0.86331 0.89954
0.75327 0.8222900000000001
0.7564 0.88013
0.8255199999999999 0.91628
0.8648600000000001 0.92542
0.7903099999999998 0.93417
0.99349 0.9878100000000001
0.91526 0.9732500000000001
0.99968 0.9943099999999999
0.99266 0.95841
0.95765 0.8894099999999999
0.91073 0.92293
0.9829600000000001 0.99336
0.99752 0.9878899999999999
0.99721 0.99567
0.9858199999999999 0.977
0.98084 0.98409
0.6840999999999999 0.8820100000000001
0.9695100000000001 0.9912000000000001
0.97184 0.99

## 🏁  Результаты

In [50]:
auc_np = np.array(auc_list0)
print("*" * 50)
print("OLD Медиана auc = {}".format(np.median(auc_np)))
print("OLD Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

auc_np = np.array(auc_list1)
print("*" * 50)
print("NEW Медиана auc = {}".format(np.median(auc_np)))
print("NEW Среднее auc = {}".format(np.mean(auc_np)))
print("*" * 50)

print("\nЭксперименты завершены!")

**************************************************
OLD Медиана auc = 0.9394549999999999
OLD Среднее auc = 0.9056947916666666
**************************************************
**************************************************
NEW Медиана auc = 0.96096
NEW Среднее auc = 0.9424775000000002
**************************************************

Эксперименты завершены!


In [52]:



tf.random.set_seed(1234)
print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'
tf.random.set_seed(1234)
print(tf.random.uniform([1], seed=1))  # generates 'A1'
print(tf.random.uniform([1], seed=1))  # generates 'A2'

tf.Tensor([0.1689806], shape=(1,), dtype=float32)
tf.Tensor([0.7539084], shape=(1,), dtype=float32)
tf.Tensor([0.1689806], shape=(1,), dtype=float32)
tf.Tensor([0.7539084], shape=(1,), dtype=float32)


In [53]:
tf.random.set_seed(1234)

@tf.function
def foo():
  a = tf.random.uniform([1], seed=1)
  b = tf.random.uniform([1], seed=1)
  return a, b
print(foo())  # prints '(A1, A1)'
tf.random.set_seed(1234)
print(foo())  # prints '(A2, A2)'

@tf.function
def bar():
  a = tf.random.uniform([1])
  b = tf.random.uniform([1])
  return a, b
print(bar())  # prints '(A1, A2)'
print(bar())  # prints '(A3, A4)'

(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.13047123], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.1689806], dtype=float32)>)
(<tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.6087816], dtype=float32)>, <tf.Tensor: shape=(1,), dtype=float32, numpy=array([0.7539084], dtype=float32)>)


In [54]:
import warnings
warnings.filterwarnings("ignore")

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import tensorflow as tf
print(np.random.randint(10))
print(np.random.randint(10))

random_seed = 123
tf.random.set_seed(random_seed)
np.random.seed(random_seed)

2
2


In [55]:
print(np.random.randint(10))
print(np.random.randint(10))
np.random.seed(random_seed)
print(np.random.randint(10))
print(np.random.randint(10))
np.random.seed(random_seed)
import numpy as np
print(np.random.randint(10))
print(np.random.randint(10))

2
2
2
2
2
2


In [56]:
import tensorflow as tf
random_seed = 123
tf.random.set_seed(random_seed)
os.environ['PYTHONHASHSEED']=str(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)
tf.compat.v1.set_random_seed(random_seed)
print(tf.random.uniform([1], seed=1))

random_seed = 123
tf.random.set_seed(random_seed)
os.environ['PYTHONHASHSEED']=str(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)
tf.compat.v1.set_random_seed(random_seed)

import tensorflow as tf
print(tf.random.uniform([1], seed=1))

tf.Tensor([0.7429987], shape=(1,), dtype=float32)
tf.Tensor([0.7429987], shape=(1,), dtype=float32)


In [57]:
import tensorflow as tf
random_seed = 123
tf.random.set_seed(random_seed)
os.environ['PYTHONHASHSEED']=str(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)
tf.compat.v1.set_random_seed(random_seed)
print(tf.random.uniform([1], seed=1))

import tensorflow as tf
random_seed = 123
tf.random.set_seed(random_seed)
os.environ['PYTHONHASHSEED']=str(random_seed)
random.seed(random_seed)
np.random.seed(random_seed)
tf.compat.v1.set_random_seed(random_seed)
print(tf.random.uniform([1], seed=1))

tf.Tensor([0.7429987], shape=(1,), dtype=float32)
tf.Tensor([0.7429987], shape=(1,), dtype=float32)
